# Lab TAT — Simulation & What‑If Modeling (03)


This notebook adds **prescriptive** analysis via discrete‑event simulation to answer questions like:
- If we add 1 technologist on the Evening shift, how much does median TAT improve?
- What SLA threshold achieves 95% compliance under current staffing?
- How sensitive are outcomes to pre‑analytical delays?

We simulate **analytical stage queues** (multi‑server) and fold in **pre** and **post** times to estimate total TAT and SLA hit rate.


In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

rng = np.random.default_rng(42)

DAYS = 14
HOURS = 24 * DAYS
SLA_MIN = 240
STAT_SHARE = 0.2
PRE_MEAN_ROUTINE = 30
PRE_MEAN_STAT = 15
POST_MEAN = 20
ANALYTICAL_MEAN = 60

arrivals_per_hour_day = np.array([
    10, 8, 6, 5, 5, 6, 8, 10, 12, 14, 16, 18,
    18, 18, 18, 20, 22, 24, 26, 24, 20, 16, 12, 10
], dtype=float)

techs_per_hour_day = np.array([
    2, 2, 2, 2, 2, 3, 3, 4, 4, 4, 4, 4,
    4, 4, 4, 4, 5, 5, 5, 4, 3, 3, 2, 2
], dtype=int)

def repeat_profile(profile, days=DAYS):
    return np.tile(profile, days)

ARRIVALS_PER_HOUR = repeat_profile(arrivals_per_hour_day, DAYS)
TECHS_PER_HOUR = repeat_profile(techs_per_hour_day, DAYS)


In [ ]:

def poisson_process_counts(lam, rng):
    return rng.poisson(lam)

def generate_arrival_times(counts, rng):
    arrival_times = []
    for h, c in enumerate(counts):
        if c <= 0:
            continue
        minutes = rng.uniform(0, 60, size=c) + h*60
        arrival_times.extend(minutes.tolist())
    arrival_times = np.array(sorted(arrival_times))
    return arrival_times

def sample_pre_times(n, rng):
    routine_n = int(np.round(n*(1-STAT_SHARE)))
    stat_n = n - routine_n
    pre_r = rng.gamma(shape=2.0, scale=PRE_MEAN_ROUTINE/2.0, size=routine_n)
    pre_s = rng.gamma(shape=2.0, scale=PRE_MEAN_STAT/2.0, size=stat_n)
    pre = np.concatenate([pre_r, pre_s])
    rng.shuffle(pre)
    return pre

def sample_post_times(n, rng):
    return rng.gamma(shape=2.0, scale=POST_MEAN/2.0, size=n)

def sample_analytical_service(n, rng):
    return rng.gamma(shape=2.0, scale=ANALYTICAL_MEAN/2.0, size=n)

def server_counts_over_time(techs_profile):
    return np.repeat(techs_profile, 60)

def simulate_queue(arrival_times_min, service_times_min, servers_per_min):
    n = len(arrival_times_min)
    waiting = np.zeros(n)
    start_times = np.zeros(n)
    end_times = np.zeros(n)

    current_minute = 0
    active_servers = int(servers_per_min[0])
    next_free = np.zeros(active_servers)

    for i, t in enumerate(arrival_times_min):
        minute = int(np.floor(t))

        if minute != current_minute:
            current_minute = minute
            target_servers = int(servers_per_min[current_minute])
            current_servers = len(next_free)
            if target_servers > current_servers:
                next_free = np.concatenate([next_free, np.full(target_servers-current_servers, current_minute)])
            elif target_servers < current_servers:
                keep_idx = np.argsort(next_free)[:target_servers]
                next_free = next_free[keep_idx]

        s_idx = np.argmin(next_free)
        start = max(t, next_free[s_idx])
        wait = max(0.0, start - t)
        end = start + service_times_min[i]

        waiting[i] = wait
        start_times[i] = start
        end_times[i] = end
        next_free[s_idx] = end

    return waiting, start_times, end_times


## Run Baseline Simulation

In [ ]:

hourly_counts = poisson_process_counts(ARRIVALS_PER_HOUR, rng)
arrivals_min = generate_arrival_times(hourly_counts, rng)
n = len(arrivals_min)

pre = sample_pre_times(n, rng)
post = sample_post_times(n, rng)
service = sample_analytical_service(n, rng)

servers_per_min = server_counts_over_time(TECHS_PER_HOUR)
wait, start_svc, end_svc = simulate_queue(arrivals_min, service, servers_per_min)

tat_total = pre + wait + service + post

median_tat = np.median(tat_total)
p95_tat = np.percentile(tat_total, 95)
sla_rate = np.mean(tat_total <= SLA_MIN) * 100

baseline = {
    "orders": int(n),
    "median_tat_min": float(median_tat),
    "p95_tat_min": float(p95_tat),
    "sla_rate_pct": float(sla_rate),
}
baseline


In [ ]:

# Total TAT distribution
fig, ax = plt.subplots(figsize=(8,4))
ax.hist(tat_total, bins=40)
ax.set_xlabel("Total TAT (minutes)")
ax.set_ylabel("Count")
ax.set_title("Baseline: Total TAT Distribution")
plt.tight_layout()
plt.show()

# Daily SLA trend (group by day index)
day_index = (arrivals_min // (60*24)).astype(int)
sla_by_day = pd.Series(tat_total <= SLA_MIN).groupby(day_index).mean()*100

fig, ax = plt.subplots(figsize=(8,4))
ax.plot(sla_by_day.index, sla_by_day.values, marker="o")
ax.set_xlabel("Day index")
ax.set_ylabel("SLA Hit Rate (%)")
ax.set_title("Baseline: Daily SLA Compliance")
ax.grid(True)
plt.tight_layout()
plt.show()


In [ ]:

def run_scenario(techs_per_hour_delta=None, sla_min=None, pre_mean_routine=None, pre_mean_stat=None):
    global PRE_MEAN_ROUTINE, PRE_MEAN_STAT, SLA_MIN

    _pre_r, _pre_s, _sla = PRE_MEAN_ROUTINE, PRE_MEAN_STAT, SLA_MIN

    if sla_min is not None:
        SLA_MIN = sla_min
    if pre_mean_routine is not None:
        PRE_MEAN_ROUTINE = pre_mean_routine
    if pre_mean_stat is not None:
        PRE_MEAN_STAT = pre_mean_stat

    techs = TECHS_PER_HOUR.copy()
    if techs_per_hour_delta is not None:
        techs = techs + techs_per_hour_delta
        techs = np.clip(techs, 1, None).astype(int)

    counts = poisson_process_counts(ARRIVALS_PER_HOUR, rng)
    arr = generate_arrival_times(counts, rng)
    m = len(arr)
    pre = sample_pre_times(m, rng)
    post = sample_post_times(m, rng)
    svc = sample_analytical_service(m, rng)
    servers_min = server_counts_over_time(techs)

    wait, s, e = simulate_queue(arr, svc, servers_min)
    tat = pre + wait + svc + post

    result = {
        "orders": int(m),
        "median_tat_min": float(np.median(tat)),
        "p95_tat_min": float(np.percentile(tat,95)),
        "sla_rate_pct": float(np.mean(tat <= SLA_MIN)*100),
    }

    PRE_MEAN_ROUTINE, PRE_MEAN_STAT, SLA_MIN = _pre_r, _pre_s, _sla
    return result


## Scenarios

In [ ]:

# +1 Evening Tech (16:00–20:00)
delta_evening = np.zeros_like(TECHS_PER_HOUR)
for d in range(DAYS):
    base = d*24
    for h in range(16, 21):
        delta_evening[base+h] = 1

sc1 = run_scenario(techs_per_hour_delta=delta_evening)

# Redistribute: -1 Night (0–5) → +1 Evening (17–22)
delta_redist = np.zeros_like(TECHS_PER_HOUR)
for d in range(DAYS):
    base = d*24
    for h in range(0, 6):
        delta_redist[base+h] = -1
    for h in range(17, 23):
        delta_redist[base+h] += 1

sc2 = run_scenario(techs_per_hour_delta=delta_redist)

# Pre-analytical improvement -20%
sc3 = run_scenario(pre_mean_routine=PRE_MEAN_ROUTINE*0.8, pre_mean_stat=PRE_MEAN_STAT*0.8)

# SLA sensitivity
sla_grid = [120, 180, 240, 300, 360]
sla_results = [run_scenario(sla_min=s) for s in sla_grid]

summary = pd.DataFrame.from_records(
    [dict(scenario="Baseline", **baseline),
     dict(scenario="+1 Evening Tech", **sc1),
     dict(scenario="Redistribute Night→Evening", **sc2),
     dict(scenario="Pre Delay -20%", **sc3)]
)
sla_df = pd.DataFrame({"SLA_min": sla_grid, "SLA_rate_pct": [r["sla_rate_pct"] for r in sla_results]})

summary, sla_df


In [ ]:

# Scenario vs median TAT
fig, ax = plt.subplots(figsize=(8,4))
ax.bar(summary["scenario"], summary["median_tat_min"])
ax.set_ylabel("Median Total TAT (min)")
ax.set_title("Scenario Comparison — Median TAT")
plt.xticks(rotation=20, ha="right")
plt.tight_layout()
plt.show()

# SLA threshold vs compliance
fig, ax = plt.subplots(figsize=(8,4))
ax.plot(sla_df["SLA_min"], sla_df["SLA_rate_pct"], marker="o")
ax.set_xlabel("SLA Threshold (minutes)")
ax.set_ylabel("SLA Hit Rate (%)")
ax.set_title("SLA Sensitivity Curve")
ax.grid(True)
plt.tight_layout()
plt.show()



## Interpreting Results (fill with your numbers)
- **+1 Evening Tech**: median TAT decreased by ~X minutes; SLA ↑ Y pp.  
- **Redistribute Night→Evening**: similar improvement without adding headcount.  
- **Pre‑analytical -20%**: Z pp improvement in SLA; suggests courier/triage optimizations can rival staffing changes.  
- **SLA sensitivity**: To hit 95% compliance, you need ~W minutes SLA or staffing uplift of ...

Summarize before/after metrics in your README.
